In [1]:
# 개방형 질문
# 진술촉진
# 구체적 질문
# 폐쇄형 질문
# 폐쇄형 질문의 형태로 질문하는 것이 중요하다. 
'''
안방, 거실, 주방, 화장실 또는 다른 곳 중에서 어디인가요?
당신의 부모님은 무슨 일이 있었는지 다른 사람이 알길 원하나요?
'''
# 폐쇄형 질문은 위와 같이 대답이 단답형이거나 예 또는 아니오로 끝나는 형태의 질문
# 예, 아니오, 잘 모르겠어요로 대답이 한정적이기에 구체적 질문의 형태는 제한한다.
# 어떻게 바꿔야할까
# 동사는 무조건 능동태로 사용하라, 형용사 제거
# 언제, 어디서, 누가, 어떻게, 왜, 무엇을 -> 해당 사항들을 알아내는 것이 목표

'\n안방, 거실, 주방, 화장실 또는 다른 곳 중에서 어디인가요?\n당신의 부모님은 무슨 일이 있었는지 다른 사람이 알길 원하나요?\n'

In [6]:
from collections import Counter

from konlpy.corpus import kolaw
from konlpy.tag import Hannanum
from konlpy.utils import concordance, pprint
from symspellpy_ko import KoSymSpell, Verbosity
from matplotlib import pyplot
from konlpy.tag import *
from hanspell import spell_checker as spc
import sys

hannanum = Hannanum()
kkma = Kkma()
okt = Okt()
sym_spell = KoSymSpell()
sym_spell.load_korean_dictionary(decompose_korean=True, load_bigrams=True)

'''
text = "저부 말헤 주세오"
for suggestion in sym_spell.lookup_compound(text, max_edit_distance=2):
  print(suggestion.term, suggestion.distance, suggestion.count)

# 전부 말해 주세요

from hanspell import spell_checker as spc

fix_chk_1 = spc.check(chk_1)
'''

talk1 = ('안녕하세요! 저는 센터에서 온 직원이에요. 철수님 맞으시죠?\
        _지금부터 사실확인을 위해 질문을 할거에요.\n반드시 알고 있는 것 모두 대답해주셔야해요! 아시겠죠?\
        _최근에 누가 때린 적이 있나요?\
        _주변 사람 이었나요?' +
# 여기서 막힘, 아빠, 엄마, 간호인, 선생님, 옆집 아저씨 인물이 너무 많아진다
# 여기서만 간단한 텍스트 입력이나 aac를 활용한 입력을 사용하는게 어떨까?
# 예를 들어 질문자 쪽에서 aac 그림을 직접 선택한다 -> 예, 아니오 선택지를 질문자가 선택한 그림들 중 
# 하나 또는 여러개 또는 '잘모르겠어요' 를 고르도록 선택한다.

        '_간접적이었나요? 직접적이었나요?' +\
# 예 아니오로 끝나지 않는 문장에 대한 예외 처리
# 현재 아이디어 - 물음표가 일정 이상일 경우 재입력 권유
        '_철수님을 손이나 도구로 직접 때렸나요?\
        _그럼 아저씨가 밀거나 당겨서 다친거군요?\
        _많이 아팠겠네요. 그럼 이제부터 '
)

li = talk1.split('_')

### 불용어

In [3]:
stopwords = '아,\
휴,\
아이구,\
아이쿠,\
아이고,\
어,\
나,\
우리,\
저희,\
따라,\
의해,\
는,\
을,\
를,\
에,\
의,\
가,\
으로,\
로,\
에게,\
뿐이다,\
의거하여,\
근거하여,\
입각하여,\
기준으로,\
예하면,\
예를 들면,\
예를 들자면,\
소인,\
소생,\
저희,\
지말고,\
하지마,\
하지마라,\
다른,\
물론,\
또한,\
그리고,\
비길수 없다,\
해서는 안된다,\
뿐만 아니라,\
만이 아니다,\
만은 아니다,\
막론하고,\
관계없이,\
그치지 않다,\
그러나,\
그런데,\
하지만,\
든간에,\
논하지 않다,\
따지지 않다,\
설사,\
비록,\
더라도,\
아니면,\
만 못하다,\
하는 편이 낫다,\
불문하고,\
향하여,\
향해서,\
향하다,\
쪽으로,\
틈타,\
이용하여,\
타다,\
오르다,\
제외하고,\
이 외에,\
이 밖에,\
하여야,\
비로소,\
한다면 몰라도,\
외에도,\
이곳,\
여기,\
부터,\
기점으로,\
따라서,\
할 생각이다,\
하려고하다,\
이리하여,\
그리하여,\
그렇게 함으로써,\
하지만,\
일때,\
할때,\
앞에서,\
중에서,\
보는데서,\
으로써,\
로써,\
까지,\
해야한다,\
일것이다,\
반드시,\
할줄알다,\
할수있다,\
할수있어,\
임에 틀림없다,\
한다면,\
등,\
등등,\
제,\
겨우,\
단지,\
다만,\
할뿐,\
딩동,\
댕그,\
대해서,\
대하여,\
대하면,\
훨씬,\
얼마나,\
얼마만큼,\
얼마큼,\
남짓,\
여,\
얼마간,\
약간,\
다소,\
좀,\
조금,\
다수,\
몇,\
얼마,\
지만,\
하물며,\
또한,\
그러나,\
그렇지만,\
하지만,\
이외에도,\
대해 말하자면,\
뿐이다,\
다음에,\
반대로,\
반대로 말하자면,\
이와 반대로,\
바꾸어서 말하면,\
바꾸어서 한다면,\
만약,\
그렇지않으면,\
까악,\
툭,\
딱,\
삐걱거리다,\
보드득,\
비걱거리다,\
꽈당,\
응당,\
해야한다,\
에 가서,\
각,\
각각,\
여러분,\
각종,\
각자,\
제각기,\
하도록하다,\
와,\
과,\
그러므로,\
그래서,\
고로,\
한 까닭에,\
하기 때문에,\
거니와,\
이지만,\
대하여,\
관하여,\
관한,\
과연,\
실로,\
아니나다를가,\
생각한대로,\
진짜로,\
한적이있다,\
하곤하였다,\
하,\
하하,\
허허,\
아하,\
거바,\
와,\
오,\
왜,\
어째서,\
무엇때문에,\
어찌,\
하겠는가,\
무슨,\
어디,\
어느곳,\
더군다나,\
하물며,\
더욱이는,\
어느때,\
언제,\
야,\
이봐,\
어이,\
여보시오,\
흐흐,\
흥,\
휴,\
헉헉,\
헐떡헐떡,\
영차,\
여차,\
어기여차,\
끙끙,\
아야,\
앗,\
아야,\
콸콸,\
졸졸,\
좍좍,\
뚝뚝,\
주룩주룩,\
솨,\
우르르,\
그래도,\
또,\
그리고,\
바꾸어말하면,\
바꾸어말하자면,\
혹은,\
혹시,\
답다,\
및,\
그에 따르는,\
때가 되어,\
즉,\
지든지,\
설령,\
가령,\
하더라도,\
할지라도,\
일지라도,\
지든지,\
몇,\
거의,\
하마터면,\
인젠,\
이젠,\
된바에야,\
된이상,\
만큼' + \
'어찌됏든,\
그위에,\
게다가,\
점에서 보아,\
비추어 보아,\
고려하면,\
하게될것이다,\
일것이다,\
비교적,\
좀,\
보다더,\
비하면,\
시키다,\
하게하다,\
할만하다,\
의해서,\
연이서,\
이어서,\
잇따라,\
뒤따라,\
뒤이어,\
결국,\
의지하여,\
기대여,\
통하여,\
자마자,\
더욱더,\
불구하고,\
얼마든지,\
마음대로,\
주저하지 않고,\
곧,\
즉시,\
바로,\
당장,\
하자마자,\
밖에 안된다,\
하면된다,\
그래,\
그렇지,\
요컨대,\
다시 말하자면,\
바꿔 말하면,\
즉,\
구체적으로,\
말하자면,\
시작하여,\
시초에,\
이상,\
허,\
헉,\
허걱,\
바와같이,\
해도좋다,\
해도된다,\
게다가,\
더구나,\
하물며,\
와르르,\
팍,\
퍽,\
펄렁,\
동안,\
이래,\
하고있었다,\
이었다,\
에서,\
로부터,\
까지,\
예하면,\
했어요,\
해요,\
함께,\
같이,\
더불어,\
마저,\
마저도,\
양자,\
모두,\
습니다,\
가까스로,\
하려고하다,\
즈음하여,\
다른,\
다른 방면으로,\
해봐요,\
습니까,\
했어요,\
말할것도 없고,\
무릎쓰고,\
개의치않고,\
하는것만 못하다,\
하는것이 낫다,\
매,\
매번,\
들,\
모,\
어느것,\
어느,\
로써,\
갖고말하자면,\
어디,\
어느쪽,\
어느것,\
어느해,\
어느 년도,\
라 해도,\
언젠가,\
어떤것,\
어느것,\
저기,\
저쪽,\
저것,\
그때,\
그럼,\
그러면,\
요만한걸,\
그래,\
그때,\
저것만큼,\
그저,\
이르기까지,\
할 줄 안다,\
할 힘이 있다,\
너,\
너희,\
당신,\
어찌,\
설마,\
차라리,\
할지언정,\
할지라도,\
할망정,\
할지언정,\
구토하다,\
게우다,\
토하다,\
메쓰겁다,\
옆사람,\
퉤,\
쳇,\
의거하여,\
근거하여,\
의해,\
따라,\
힘입어,\
그,\
다음,\
버금,\
두번째로,\
기타,\
첫번째로,\
나머지는,\
그중에서,\
견지에서,\
형식으로 쓰여,\
입장에서,\
위해서,\
단지,\
의해되다,\
하도록시키다,\
뿐만아니라,\
반대로,\
전후,\
전자,\
앞의것,\
잠시,\
잠깐,\
하면서,\
그렇지만,\
다음에,\
그러한즉,\
그런즉,\
남들,\
아무거나,\
어찌하든지,\
같다,\
비슷하다,\
예컨대,\
이럴정도로,\
어떻게,\
만약,\
만일,\
위에서 서술한바와같이,\
인 듯하다,\
하지 않는다면,\
만약에,\
무엇,\
무슨,\
어느,\
어떤,\
아래윗,\
조차,\
한데,\
그럼에도 불구하고,\
여전히,\
심지어,\
까지도,\
조차도,\
하지 않도록,\
않기 위하여,\
때,\
시각,\
무렵,\
시간,\
동안,\
어때,\
어떠한,\
하여금,\
네,\
예,\
우선,\
누구,\
누가 알겠는가,\
아무도,\
줄은모른다,\
줄은 몰랏다,\
하는 김에,\
겸사겸사,\
하는바,\
그런 까닭에,\
한 이유는,\
그러니,\
그러니까,\
때문에,\
그,\
너희,\
그들,\
너희들,\
타인,\
것,\
것들,\
너,\
위하여,\
공동으로,\
동시에,\
하기 위하여,\
어찌하여,\
무엇때문에,\
붕붕,\
윙윙,\
나,\
우리,\
엉엉,\
휘익,\
윙윙,\
오호,\
아하,\
어쨋든,\
만 못하다' +\
'하기보다는,\
차라리,\
하는 편이 낫다,\
흐흐,\
놀라다,\
상대적으로 말하자면,\
마치,\
아니라면,\
쉿,\
그렇지 않으면,\
그렇지 않다면,\
안 그러면,\
아니었다면,\
하든지,\
아니면,\
이라면,\
좋아,\
알았어,\
하는것도,\
그만이다,\
어쩔수 없다,\
하나,\
일,\
일반적으로,\
일단,\
한켠으로는,\
오자마자,\
이렇게되면,\
이와같다면,\
전부,\
한마디,\
한항목,\
근거로,\
하기에,\
아울러,\
하지 않도록,\
않기 위해서,\
이르기까지,\
이 되다,\
로 인하여,\
까닭으로,\
이유만으로,\
이로 인하여,\
그래서,\
이 때문에,\
그러므로,\
그런 까닭에,\
알 수 있다,\
결론을 낼 수 있다,\
으로 인하여,\
있다,\
어떤것,\
관계가 있다,\
관련이 있다,\
연관되다,\
어떤것들,\
에 대해,\
이리하여,\
그리하여,\
여부,\
하기보다는,\
하느니,\
하면 할수록,\
운운,\
이러이러하다,\
하구나,\
하도다,\
다시말하면,\
다음으로,\
에 있다,\
에 달려 있다,\
우리,\
우리들,\
오히려,\
하기는한데,\
어떻게,\
어떻해,\
어찌됏어,\
어때,\
어째서,\
본대로,\
자,\
이,\
이쪽,\
여기,\
이것,\
이번,\
이렇게말하자면,\
이런,\
이러한,\
이와 같은,\
요만큼,\
요만한 것,\
얼마 안 되는 것,\
이만큼,\
이 정도의,\
이렇게 많은 것,\
이와 같다,\
이때,\
이렇구나,\
것과 같이,\
끼익,\
삐걱,\
따위,\
와 같은 사람들,\
부류의 사람들,\
왜냐하면,\
중의하나,\
오직,\
오로지,\
에 한하다,\
하기만 하면,\
도착하다,\
까지 미치다,\
도달하다,\
정도에 이르다,\
할 지경이다,\
결과에 이르다,\
관해서는,\
여러분,\
하고 있다,\
한 후,\
혼자,\
자기,\
자기집,\
자신,\
우에 종합한것과같이,\
총적으로 보면,\
총적으로 말하면,\
총적으로,\
대로 하다,\
으로서,\
참,\
그만이다,\
할 따름이다,\
쿵,\
탕탕,\
쾅쾅,\
둥둥,\
봐,\
봐라,\
아이야,\
아니,\
와아,\
응,\
아이,\
참나,\
년,\
월,\
일,\
령,\
영,\
일,\
이,\
삼,\
사,\
오,\
육,\
륙,\
칠,\
팔,\
구,\
이천육,\
이천칠,\
이천팔,\
이천구,\
하나,\
둘,\
셋,\
넷,\
다섯,\
여섯,\
일곱,\
여덟,\
아홉,\
령,\
영'

In [4]:
list_stop = stopwords.split(',')

### 조사 예시

In [5]:
from pyjosa.josa import Josa as js

subject = '피해자 분'
target = '폭행'
verb = '당하다'

full_string = f'{js.get_full_string(subject, "는")} {js.get_full_string(target, "를")} ' + verb

print(full_string)

피해자 분은 폭행을 당하다


### 변환기 실험

In [42]:
# list_stop : 불용어 리스트
# global stop, YNq

# 예, 아니오, 잘 모르겠어요로 답이 가능한지 확인
warn_YN = '예 또는 아니오로 답할 수 있는 문장이 아닙니다!'

def __chkYN__(question):

    if question == 'end':
        print('End test')
        return True, 'good'
    elif not question or question.count('?') >= 2 or question[-1] != '?':
        print(warn_YN)
        return False, 'ng'
    else:
        fix = spc.check(question)
        print(fix.checked)
        return False, 'good'

# 동사의 능동태 확인
def __chkvoice__(question):
    tmp = kkma.pos(question)
    result = []
    for item in tmp:
        if 'VV' in item:
            result.append(tmp.index(item))

    for i in result:
        if tmp[i+1][1] == 'ECS':
            print('능동태를 사용해주세요\n능동태는 다음과 같습니다 :\n예시 넣기!')
            return True
        else:
            return False

# 답변 기능
def __ans__(YN):
    if YN == '네':
        print('네')
    elif YN == '아니오':
        print('아니오')
    else:
        print('질문을 다시 해주세요')


In [45]:
while(True):
    str_line = input()
    stop = False
    Vq = False
    YNq = 'ng'

    stop, YNq = __chkYN__(str_line)
    Vq = __chkvoice__(str_line)

    if stop and YNq == 'good':
        break
    elif not stop and YNq == 'ng':
        continue
    elif Vq:
        continue
    else:
        ans = input()
        __ans__(ans)

남겨졌나요?
능동태를 사용해주세요
능동태는 다음과 같습니다 :

End test


## test

In [ ]:
for i in li:
  result = []
  print(okt.pos(i))
  for w in okt.morphs(i):
    if w not in list_stop:
      result.append(w)
  print('불용어 제거 결과')
  print(result)
  print('\n')

[('안녕하세요', 'Adjective'), ('!', 'Punctuation'), ('저', 'Noun'), ('는', 'Josa'), ('센터', 'Noun'), ('에서', 'Josa'), ('온', 'Noun'), ('직원', 'Noun'), ('이에요', 'Josa'), ('.', 'Punctuation'), ('철수', 'Noun'), ('님', 'Suffix'), ('맞으시죠', 'Verb'), ('?', 'Punctuation')]
불용어 제거 결과
['안녕하세요', '!', '저', '센터', '온', '직원', '이에요', '.', '철수', '님', '맞으시죠', '?']


[('지금', 'Noun'), ('부터', 'Josa'), ('사실', 'Noun'), ('확인', 'Noun'), ('을', 'Josa'), ('위해', 'Noun'), ('질문', 'Noun'), ('을', 'Josa'), ('할거에요', 'Verb'), ('.', 'Punctuation'), ('\n', 'Foreign'), ('반드시', 'Noun'), ('알', 'Noun'), ('고', 'Josa'), ('있는', 'Adjective'), ('것', 'Noun'), ('모두', 'Noun'), ('대답', 'Noun'), ('해주셔', 'Verb'), ('야해요', 'Adjective'), ('!', 'Punctuation')]
불용어 제거 결과
['지금', '사실', '확인', '위해', '질문', '할거에요', '.', '\n', '알', '고', '있는', '대답', '해주셔', '야해요', '!']


[('최근', 'Noun'), ('에', 'Josa'), ('누가', 'Noun'), ('때린', 'Verb'), ('적', 'Noun'), ('이', 'Josa'), ('있나요', 'Adjective'), ('?', 'Punctuation')]
불용어 제거 결과
['최근', '누가', '때린', '적', '있나요', '?']


[('주변', 'Nou

In [50]:
test = '많이 맞았나요?'
test1 = '그 사람한테 많이 맞았나요?'
test2 = '아저씨가 많이 때렸나요?'

print(okt.pos(test), '\n', kkma.pos(test))
print(okt.pos(test1), '\n', kkma.pos(test1))
print(okt.pos(test2), '\n', kkma.pos(test2))

[('많이', 'Adverb'), ('맞았나요', 'Verb'), ('?', 'Punctuation')] 
 [('많이', 'MAG'), ('맞', 'VV'), ('았', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]
[('그', 'Noun'), ('사람', 'Noun'), ('한테', 'Josa'), ('많이', 'Adverb'), ('맞았나요', 'Verb'), ('?', 'Punctuation')] 
 [('그', 'MDT'), ('사람', 'NNG'), ('한테', 'JKO'), ('많이', 'MAG'), ('맞', 'VV'), ('았', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]
[('아저씨', 'Noun'), ('가', 'Josa'), ('많이', 'Adverb'), ('때렸나요', 'Verb'), ('?', 'Punctuation')] 
 [('아저씨', 'NNG'), ('가', 'JKS'), ('많이', 'MAG'), ('때리', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]


In [41]:
test2 = '잡히었나요? 잡았나요? 잡아졌나요?'

t = kkma.pos(test2)
print(t)
result = []
for item in t:
    if 'VV' in item:
        result.append(t.index(item))

print(result)

print('\n')
test3 = '때려졌나요? 때렸나요?'
print(kkma.pos(test3))
test4 = '뚫리었나요? 뚫었나요'
print(kkma.pos(test4))
test5 = '가지었나요? 가졌나요? 가져갔나요? 가지고 갔나요'
print(kkma.pos(test5))
test6 = '남기고 갔나요? 남기어졌나요? 남겨졌나요?'
print(kkma.pos(test6))


[('잡히', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('잡', 'VV'), ('았', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('잡', 'VV'), ('아', 'ECD'), ('지', 'VXV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]
[0, 4, 4]


[('때리', 'VV'), ('어', 'ECS'), ('지', 'VXV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('때리', 'VXV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]
[('뚫리', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('뚫', 'VV'), ('었', 'EPT'), ('나요', 'EFQ')]
[('가지', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('가지', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('가져가', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('가지', 'VV'), ('고', 'ECE'), ('가', 'VV'), ('었', 'EPT'), ('나요', 'EFQ')]
[('남기', 'VV'), ('고', 'ECE'), ('가', 'VV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('남기', 'VV'), ('어', 'ECD'), ('지', 'VXV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF'), ('남기', 'VV'), ('어', 'ECS'), ('지', 'VXV'), ('었', 'EPT'), ('나요', 'EFQ'), ('?', 'SF')]


In [46]:
from pyjosa.josa import Josa as js

subject = '피해자 분'
target = '폭행'
verb = '당하다'

full_string = f'{js.get_full_string(subject, "는")} {js.get_full_string(target, "를")} ' + verb

print(full_string)

피해자 분은 폭행을 당하다


In [ ]:
for i in li:
    result = []
    print('꼬꼬마: ', kkma.nouns(i))
    print(kkma.pos(i))
    print('\n')
    for w in kkma.morphs(i):
        if w not in list_stop:
            result.append(w)
    print('불용어 제거 결과')
    print(result)
    print('\n')

꼬꼬마:  ['안녕', '저', '센터', '직원', '철수']
[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('!', 'SF'), ('저', 'NP'), ('는', 'JX'), ('센터', 'NNG'), ('에서', 'JKM'), ('오', 'VV'), ('ㄴ', 'ETD'), ('직원', 'NNG'), ('이', 'VCP'), ('에요', 'EFN'), ('.', 'SF'), ('철수', 'NNG'), ('님', 'XSN'), ('맞', 'VV'), ('으시', 'EPH'), ('죠', 'EFN'), ('?', 'SF')]


불용어 제거 결과
['안녕', '세요', '!', '저', '센터', 'ㄴ', '직원', '에요', '.', '철수', '님', '맞', '으시', '죠', '?']


꼬꼬마:  ['지금', '사실', '사실확인', '확인', '질문', '할거', '대답']
[('지금', 'NNG'), ('부터', 'JX'), ('사실', 'NNG'), ('확인', 'NNG'), ('을', 'JKO'), ('위하', 'VV'), ('어', 'ECS'), ('질문', 'NNG'), ('을', 'JKO'), ('할거', 'NNG'), ('에', 'JKM'), ('요', 'JX'), ('.', 'SF'), ('반드시', 'MAG'), ('알', 'VV'), ('고', 'ECE'), ('있', 'VXV'), ('는', 'ETD'), ('것', 'NNB'), ('모두', 'MAG'), ('대답', 'NNG'), ('하', 'XSV'), ('어', 'ECS'), ('주', 'VXV'), ('시', 'EPH'), ('어야', 'ECD'), ('하', 'VV'), ('어요', 'EFN'), ('!', 'SF')]


불용어 제거 결과
['지금', '사실', '확인', '위하', '질문', '할거', '요', '.', '알', '고', '있', '대답', '주', '시', '어야', '어요', '!']


꼬꼬마:  ['최근',